In [ ]:
import numpy as np
from scipy.linalg import fractional_matrix_power
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import animation

This notebook will help you with the numerical calculations for **Segway Tours**.

## Part (b): Convert linear continuous time system to linear discrete time system

In [ ]:
# define constants
m = 1
M = 10
g = 10
l = 1
k = 0.1
Delta = 1

# define A and \vec{b} for linearized system
A = np.array([[0, 1, 0, 0],
              [0, -k/M, -(m*g)/M, 0],
              [0, 0, 0, 1],
              [0, k/(M*l), ((M+m)*g)/(M*l), 0]])
b = np.array([[0], [1/M], [0], [-1/(M*l)]])

In [ ]:
# Eigendecomposition of A
eigs, V = np.linalg.eig(A)
V_inv = np.linalg.inv(V)

# Convert to discrete time parameters A_d and \vec{b}_d
Lambda_d = np.diag(np.exp(eigs * Delta))
A_d = V @ Lambda_d @ V_inv

num = np.exp(eigs * Delta) - 1
diag = np.ones_like(eigs) * Delta
for idx, ele in enumerate(eigs):
    if ele:
        diag[idx] = num[idx] / ele
M_d = np.diag(diag)
b_d = V @ M_d @ V_inv @ b

print(f'A_d={A_d}')
print(f'b_d={b_d}')

## Part (c): Controllability of linear discrete time system

In [ ]:
Ab = A_d @ b_d
A2b = A_d @ Ab
A3b = A_d @ A2b
### BEGIN STUDENT
# HINT: remember from Note 11 page 3 that for the system given by x[n+1] = A x[n] + B u[n]
# the controllability matrix is C = [B AB A^2B ... A^(n-1)B].
# Adapt that result to the current system. You may want to use the Numpy function np.hstack().
# Please pay attention to the order of the columns of C. If you flip the order, the animation code will break.
C = ...
### END STUDENT

rank_C = np.linalg.matrix_rank(C)
print(f'Rank of controllability matrix is {rank_C}')
print(f'C={C}')

## Parts (d) and (e): Find input control sequence for bringing given initial state to upright position at rest

In [ ]:
state_final = np.array([[0], [0], [0], [0]])
state_initial = np.array([[-2], [3.1], [0.3], [-0.6]])  # part (d): linearization is valid
#state_initial = np.array([[-2], [3.1], [3.3], [-0.6]])  # part (e): linearization is not valid
A4 = A_d @ A_d @ A_d @ A_d

### BEGIN STUDENT
# HINT: Look at Note 11 page 5
u_d = ...
### END STUDENT
print(f'u_d={u_d}')

Note that, because of the column order we chose for the controllability matrix, the control input vector $u_d$ is arranged as $u_d = \begin{bmatrix} u_d[3] \\ u_d[2] \\ u_d[1] \\ u_d[0] \end{bmatrix}$.

## Set up simulation (adapted from old 16A code)

### Preamble

This function will take care of animating the segway. You do not have to understand the code in this cell for this HW.

In [ ]:
# frames per second in simulation
fps = 20
# length of the segway arm/stick
stick_length = 20.

def animate_segway(t, states, controls, length):
    #Animates the segway
    
    # Set up the figure, the axis, and the plot elements we want to animate
    fig = plt.figure()
    
    # some config
    segway_width = 10.4
    segway_height = 10.2
    
    # x coordinate of the segway stick
    segwayStick_x = np.add(states[0, :], length * np.sin(states[2, :]))
    segwayStick_y = length * np.cos(states[2, :])
    
    # set the limits
    xmin = min(np.around(states[0, :].min() - segway_width / 2.0, 1), np.around(segwayStick_x.min(), 1))
    xmax = max(np.around(states[0, :].max() + segway_width / 2.0, 1), np.around(segwayStick_x.max(), 1))
    
    # create the axes
    ax = plt.axes(xlim=(xmin-length, xmax+length), ylim=(-length * 1.2, length * 1.2), aspect='equal')
    
    # display the current time
    time_text = ax.text(0.05, 0.9, 'time', transform=ax.transAxes)
    
    # display the current control
    control_text = ax.text(0.05, 0.8, 'control', transform=ax.transAxes)
    
    # create rectangle for the segway
    rect = Rectangle([states[0, 0] - segway_width / 2.0, -segway_height / 2],
        segway_width, segway_height, fill=True, color='gold', ec='blue')
    ax.add_patch(rect)
    
    # blank line for the stick with o for the ends
    stick_line, = ax.plot([], [], lw=2, marker='o', markersize=6, color='blue')

    # vector for the control (force)
    force_vec = ax.quiver([],[],[],[],angles='xy',scale_units='xy',scale=1)

    # initialization function: plot the background of each frame
    def init():
        time_text.set_text('')
        control_text.set_text('')
        rect.set_xy((0.0, 0.0))
        stick_line.set_data([], [])
        return time_text, rect, stick_line, control_text

    # animation function: update the objects
    def animate(i):
        time_text.set_text('time = {:2.2f}'.format(t[i]))
        control_text.set_text('force = {:2.3f}'.format(controls[i]))
        rect.set_xy((states[0, i] - segway_width / 2.0, -segway_height / 2))
        stick_line.set_data([states[0, i], segwayStick_x[i]], [0, segwayStick_y[i]])
        return time_text, rect, stick_line, control_text

    # call the animator function
    anim = animation.FuncAnimation(fig, animate, frames=len(t), init_func=init,
            interval=1000/fps, blit=False, repeat=False)
    return anim
    #plt.show()

### Simulation: Rerun this cell whenever you change any state or constants for the discrete time system

You do not have to understand the code in this cell for this HW.

In [ ]:
controls = np.flip(np.squeeze(u_d))

# Add an extra couple of seconds to the simulation after the input controls with no control.
# The effect of this is just to show how the system will continue after the controller "stops controlling"
controls = np.append(controls, [0, 0])

# number of steps in the simulation
nr_steps = controls.shape[0]
nr_states = A_d.shape[0]

# We now compute finer dynamics and control vectors for smoother visualization
Afine = fractional_matrix_power(A_d, (1/fps))
Asum = np.eye(nr_states)
for idx in range(1, fps):
    Asum = Asum + np.linalg.matrix_power(Afine, idx)
    
bfine = np.linalg.inv(Asum) @ b_d

# We also expand the controls in the "intermediate steps" (only for visualization)
controls_final = np.outer(controls, np.ones(fps)).flatten()
controls_final = np.append(controls_final, [0])

# We compute all the states starting from x0 and using the controls
states = np.empty([nr_states, fps*(nr_steps)+1])
states[:, [0]] = state_initial
for stepId in range(1, fps*(nr_steps)+1):
    states[:, [stepId]] = Afine @ states[:, [stepId-1]] + bfine * controls_final[stepId-1]
    
# Now create the time vector for simulation
t = np.linspace(1/fps, nr_steps, fps*(nr_steps), endpoint=True)
t = np.append([0], t)

### Visualization for discrete time system

In [ ]:
%matplotlib nbagg
anim = animate_segway(t, states, controls_final, stick_length)
anim

Since the system is controllable, the segway should stabilize to upright position at rest using the correct input sequence $u_d$. However, the behavior of the segway looks more realistic in part (d) rather than the unexpected rotations in part (e). 
This is because the linearization approximation is valid only for small $\theta$ and $\frac{d\theta}{dt}$ in part (d), but not for the larger $\theta$ in part (e). Hence the discrete time linear model is a good approximation of the original continuous time non-linear system only for small deviations around the upright position at rest. In order to properly model the system for any deviation, we will have to use a discrete time non-linear model. 

It is important to understand that this is a limitation of linearizing a non-linear system, regardless of whether we analyze the system in continuous or discrete time domain. Analyzing the system in continuous time instead of discretizing won't remove the linearization approximation issue. To help convince ourselves, let's compare the original continuous time non-linear system with its continuous time linear model. Deriving the control inputs in the continuous time case is out of scope of 16B, so let's look at the simpler case of the segway stabilizing to the steady hanging position at rest in the absence of any input.

## Part (f): Continuous time Non Linear vs Linear Model

In [ ]:
def next_state(cur_state, u, dt, linearized):
    '''Calculates and returns the next state based on the current state and applied input for continuous time system. 
    Has option for linear and non-linear state variable calculation.
    Parameters
    ----------
    cur_state: np.ndarray
        (4, 1) array respresenting current state
    u: float
        current control input
    dt: float
        time interval
    linearized: bool
        True to use linear model, False to use non-linear model
    
    Returns
    -------
    next_state: np.ndarray
        (4, 1) array respresenting current state
    '''
    # Unpack state variables
    x = cur_state[0, 0]
    x_dot = cur_state[1, 0]
    theta = cur_state[2, 0]
    theta_dot = cur_state[3, 0]
    
    if linearized:
        # for linear model, use dx/dt = Ax + bu
        _temp = A @ cur_state + b * u
        acceleration = _temp[1, 0]
        ang_acceleration = _temp[3, 0]
    else:
        # Calculate angular and linear acceleration
        den = M / m + (np.sin(theta)) ** 2
        acceleration = (u / m + (theta_dot ** 2) * l * np.sin(theta) - g * np.sin(theta) * np.cos(theta) - k / m * x_dot) / den
        ang_acceleration = (-u / m * np.cos(theta) - (theta_dot ** 2) * l * np.cos(theta) * np.sin(theta) + (M/m + 1) * g * np.sin(theta) + k / m * x_dot * np.cos(theta)) / (l * den)

    # Update angular and linear velocities 
    vel_new = x_dot + acceleration * dt
    ang_vel_new = theta_dot + ang_acceleration * dt

    # Update angular and linear positions
    pos_new = x + vel_new * dt
    ang_new = theta + ang_vel_new * dt

    # Make sure angle is always between -pi and pi
    while ang_new < -np.pi:
        ang_new += 2 * np.pi
    while (ang_new > np.pi):
        ang_new -= 2 * np.pi

    # Return next state
    next_state = np.array([[pos_new], [vel_new], [ang_new], [ang_vel_new]]) 
    return next_state

### Simulation: Rerun this cell whenever you change any state or constants for the continuous time system

You do not have to understand the code in this cell for this HW. Just toggle the $\texttt{linearized}$ flag.

In [ ]:
linearized = False  # toggle between True and False

nr_steps = 20
nr_states = A.shape[0]

# We compute all the states starting from x0 and using the controls
states = np.empty([nr_states, fps*(nr_steps)+1])
states[:, [0]] = np.array([[0], [0], [3], [0]]);
for stepId in range(1, fps*(nr_steps)+1):
    states[:, [stepId]] = next_state(states[:, [stepId-1]], 0, 1/fps, linearized)

# Now create the time vector for simulation
t = np.linspace(1/fps, nr_steps, fps*(nr_steps), endpoint=True)
t = np.append([0], t)

### Visualization for continuous time system

In [ ]:
%matplotlib nbagg
anim = animate_segway(t, states, np.zeros(fps*(nr_steps)+1), stick_length)
anim

Changelog:
- Spring 21: Created by Ayan Biswas, Daniel Abraham
- Fall 21: Merged continuous time system in same notebook, Ayan Biswas